# Politecnico Malvinas Argentinas- PDH: Preprocesamiento, Normalización y Técnicas Básicas

In [2]:
# Instalación de librerías necesarias (si es necesario en Colab)
!pip install nltk spacy scikit-learn thefuzz stop_words
!pip install fuzzywuzzy python-Levenshtein

import re
import nltk
import spacy
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eecab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eecab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Cargar modelo en español de spaCy
!python -m spacy download es_core_news_md
nlp = spacy.load("es_core_news_md")

     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     ---------------------------------------- 0.5/42.3 MB 5.6 MB/s eta 0:00:08
     - -------------------------------------- 1.6/42.3 MB 5.2 MB/s eta 0:00:08
     -- ------------------------------------- 2.9/42.3 MB 5.6 MB/s eta 0:00:08
     --- ------------------------------------ 3.9/42.3 MB 5.5 MB/s eta 0:00:08
     ---- ----------------------------------- 5.0/42.3 MB 5.6 MB/s eta 0:00:07
     ----- ---------------------------------- 6.0/42.3 MB 5.3 MB/s eta 0:00:07
     ------ --------------------------------- 6.8/42.3 MB 5.0 MB/s eta 0:00:08
     ------ --------------------------------- 7.3/42.3 MB 4.8 MB/s eta 0:00:08
     ------- -------------------------------- 7.9/42.3 MB 4.5 MB/s eta 0:00:08
     -------- ------------------------------- 8.7/42.3 MB 4.3 MB/s eta 0:00:08
     -------- ------------------------------- 9.2/42.3 MB 4.2 MB/s eta 0:00:08
     --------- ------------------------------ 10.0/42.3 MB 

# 1. Preprocesamiento de texto

# Tokenizacion

### Reemplazo del dataset y adaptación del código


In [5]:
corpus = [
    "Me encanta aprender procesamiento de lenguaje natural.",
    "Los modelos como BERT y GPT han cambiado todo.",
    "Esta clase del Politécnico es muy interesante y desafiante.",
    "La lematización y la vectorización son técnicas clave.",
    "Quiero crear un chatbot inteligente para atención al cliente."
]
corpus = [frase.lower() for frase in corpus]
corpus


['me encanta aprender procesamiento de lenguaje natural.',
 'los modelos como bert y gpt han cambiado todo.',
 'esta clase del politécnico es muy interesante y desafiante.',
 'la lematización y la vectorización son técnicas clave.',
 'quiero crear un chatbot inteligente para atención al cliente.']

In [6]:
import spacy

# Cargar modelo spaCy en español
nlp = spacy.load("es_core_news_md")

In [7]:
# Tokenización: se convierte cada frase en una lista de tokens
tokens_por_frase = []

for frase in corpus:
    doc = nlp(frase.lower())
    tokens = [token.text for token in doc if not token.is_punct and not token.is_space]
    tokens_por_frase.append(tokens)

# Mostrar tokens por frase
for i, tokens in enumerate(tokens_por_frase):
    print(f"Frase {i+1}: {tokens}")


Frase 1: ['me', 'encanta', 'aprender', 'procesamiento', 'de', 'lenguaje', 'natural']
Frase 2: ['los', 'modelos', 'como', 'bert', 'y', 'gpt', 'han', 'cambiado', 'todo']
Frase 3: ['esta', 'clase', 'del', 'politécnico', 'es', 'muy', 'interesante', 'y', 'desafiante']
Frase 4: ['la', 'lematización', 'y', 'la', 'vectorización', 'son', 'técnicas', 'clave']
Frase 5: ['quiero', 'crear', 'un', 'chatbot', 'inteligente', 'para', 'atención', 'al', 'cliente']


## Tokenización con spaCy

En esta sección se reemplazó la tokenización original (realizada previamente con NLTK o código manual) por un enfoque más preciso y robusto utilizando `spaCy`.

Se utiliza un nuevo corpus generado artificialmente con frases relacionadas al procesamiento de lenguaje natural. Cada frase es procesada de forma individual para extraer tokens válidos. Durante este preprocesamiento se aplican los siguientes pasos:

- Conversión del texto a minúsculas.
- Eliminación de signos de puntuación y espacios en blanco.
- Tokenización conservando la estructura frase a frase.

Esta estrategia permite una tokenización más limpia y lista para las siguientes etapas del procesamiento de lenguaje natural, manteniendo la coherencia en el corpus y facilitando análisis posteriores como lematización o vectorización.


# 2. Técnicas de NLP con NLTK y spaCy

In [8]:
# 2. Técnicas de NLP con spaCy y similitud de textos

import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from thefuzz import fuzz
from nltk.metrics import edit_distance
from pyjarowinkler import distance as jwdist

In [9]:
# Cargar modelo spaCy
nlp = spacy.load("es_core_news_md")

In [10]:
# Tokenización y stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eecab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eecab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
stop_words = set(stopwords.words('spanish'))

In [12]:
# Preprocesamiento y lematización
lemmatized_corpus = []
filtered_words = []

for frase in corpus:
    doc = nlp(frase)
    palabras = [token.text for token in doc if not token.is_punct and not token.is_space]
    lemas = [token.lemma_ for token in doc if token.text.lower() not in stop_words and not token.is_punct and not token.is_space]
    lemmatized_corpus.append(" ".join(lemas))
    filtered_words.extend(lemas)

print("\nLematización por frase:")
for i, frase in enumerate(lemmatized_corpus):
    print(f"Frase {i+1}: {frase}")



Lematización por frase:
Frase 1: encantar aprender procesamiento lenguaje natural
Frase 2: modelo bert gpt cambiar
Frase 3: clase politécnico interesante desafiante
Frase 4: lematización vectorización técnico clave
Frase 5: querer crear chatbot inteligente atención cliente


In [13]:
# Distancias entre strings (fuzzy, edit y jaro-winkler)
print("\nSimilitud entre frases del corpus:")
for i in range(len(corpus)):
    for j in range(i+1, len(corpus)):
        fz = fuzz.ratio(corpus[i], corpus[j])
        ed = edit_distance(corpus[i], corpus[j])
        jw = jwdist.get_jaro_distance(corpus[i], corpus[j])
        print(f"Frase {i+1} vs {j+1} | Fuzzy: {fz} | EditDist: {ed} | Jaro-Winkler: {jw:.2f}")


Similitud entre frases del corpus:
Frase 1 vs 2 | Fuzzy: 34 | EditDist: 45 | Jaro-Winkler: 0.42
Frase 1 vs 3 | Fuzzy: 46 | EditDist: 47 | Jaro-Winkler: 0.33
Frase 1 vs 4 | Fuzzy: 35 | EditDist: 44 | Jaro-Winkler: 0.37
Frase 1 vs 5 | Fuzzy: 42 | EditDist: 48 | Jaro-Winkler: 0.31
Frase 2 vs 3 | Fuzzy: 44 | EditDist: 44 | Jaro-Winkler: 0.38
Frase 2 vs 4 | Fuzzy: 34 | EditDist: 42 | Jaro-Winkler: 0.43
Frase 2 vs 5 | Fuzzy: 37 | EditDist: 47 | Jaro-Winkler: 0.44
Frase 3 vs 4 | Fuzzy: 37 | EditDist: 47 | Jaro-Winkler: 0.33
Frase 3 vs 5 | Fuzzy: 43 | EditDist: 48 | Jaro-Winkler: 0.31
Frase 4 vs 5 | Fuzzy: 38 | EditDist: 46 | Jaro-Winkler: 0.37


## Técnicas de NLP con spaCy y Similitud de Textos

En esta sección se aplicaron técnicas de procesamiento de lenguaje natural utilizando principalmente la biblioteca `spaCy`. El objetivo fue extraer información lingüística relevante del nuevo corpus y comparar las frases entre sí.

### Procesos aplicados:

- **Lematización**: Se transformaron las palabras a su forma base utilizando el modelo `es_core_news_md` de spaCy. Esto permite un análisis más general, evitando contar variantes gramaticales como palabras distintas.
  
- **Eliminación de stopwords**: Se descartaron palabras vacías del español (`nltk.stopwords`) que no aportan significado relevante al análisis (como "de", "y", "la").

- **Cálculo de similitud entre frases**: Se midió la similitud entre todas las combinaciones de frases del corpus mediante tres métricas:
  - `Fuzzy Ratio` (distancia aproximada entre cadenas).
  - `Edit Distance` (distancia de Levenshtein).
  - `Jaro-Winkler` (ideal para nombres y textos cortos).

### Resultados:

Se generó un conjunto lematizado del corpus y se imprimieron las frases con sus formas reducidas. Además, se mostraron comparaciones entre frases para analizar qué tan similares son entre sí según distintas métricas.

Estas técnicas permiten tener una representación más limpia y semánticamente consistente del texto, fundamental para análisis posteriores como clasificación, agrupamiento o vectorización.


# 3. Vectorización y Representación del Texto

In [15]:
texts = ["El gato come pescado", "El perro ladra en la casa"]

In [15]:
# Bolsa de Palabras (BoW)
vectorizer_bow = CountVectorizer()
X_bow = vectorizer_bow.fit_transform(lemmatized_corpus)

print("Representación BoW:")
pd.DataFrame(X_bow.toarray(), columns=vectorizer_bow.get_feature_names_out())

Representación BoW:


aprender  atención  bert  cambiar  chatbot  clase  clave  cliente  crear  \
0         1         0     0        0        0      0      0        0      0   
1         0         0     1        1        0      0      0        0      0   
2         0         0     0        0        0      1      0        0      0   
3         0         0     0        0        0      0      1        0      0   
4         0         1     0        0        1      0      0        1      1   

   desafiante  ...  interesante  lematización  lenguaje  modelo  natural  \
0           0  ...            0             0         1       0        1   
1           0  ...            0             0         0       1        0   
2           1  ...            1             0         0       0        0   
3           0  ...            0             1         0       0        0   
4           0  ...            0             0         0       0        0   

   politécnico  procesamiento  querer  técnico  vectorización  
0            0              1       0        0              0  
1            0              0       0        0              0  
2            1              0       0        0              0  
3            0              0       0        1              1  
4            0              0       1        0              0  

[5 rows x 23 columns]

In [16]:

# TF-IDF
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(lemmatized_corpus)

print("Representación TF-IDF:")
pd.DataFrame(X_tfidf.toarray(), columns=vectorizer_tfidf.get_feature_names_out())

Representación TF-IDF:


aprender  atención  bert  cambiar   chatbot  clase  clave   cliente  \
0  0.447214  0.000000   0.0      0.0  0.000000    0.0    0.0  0.000000   
1  0.000000  0.000000   0.5      0.5  0.000000    0.0    0.0  0.000000   
2  0.000000  0.000000   0.0      0.0  0.000000    0.5    0.0  0.000000   
3  0.000000  0.000000   0.0      0.0  0.000000    0.0    0.5  0.000000   
4  0.000000  0.408248   0.0      0.0  0.408248    0.0    0.0  0.408248   

      crear  desafiante  ...  interesante  lematización  lenguaje  modelo  \
0  0.000000         0.0  ...          0.0           0.0  0.447214     0.0   
1  0.000000         0.0  ...          0.0           0.0  0.000000     0.5   
2  0.000000         0.5  ...          0.5           0.0  0.000000     0.0   
3  0.000000         0.0  ...          0.0           0.5  0.000000     0.0   
4  0.408248         0.0  ...          0.0           0.0  0.000000     0.0   

    natural  politécnico  procesamiento    querer  técnico  vectorización  
0  0.447214          0.0       0.447214  0.000000      0.0            0.0  
1  0.000000          0.0       0.000000  0.000000      0.0            0.0  
2  0.000000          0.5       0.000000  0.000000      0.0            0.0  
3  0.000000          0.0       0.000000  0.000000      0.5            0.5  
4  0.000000          0.0       0.000000  0.408248      0.0            0.0  

[5 rows x 23 columns]

In [17]:
# N-gramas (bigrams y trigrams)
vectorizer_ngrams = CountVectorizer(ngram_range=(2, 3), min_df=1)
X_ngrams = vectorizer_ngrams.fit_transform(lemmatized_corpus)
print("Representación con N-gramas (2-3):")
display(pd.DataFrame(X_ngrams.toarray(), columns=vectorizer_ngrams.get_feature_names_out()))


Representación con N-gramas (2-3):


aprender procesamiento  aprender procesamiento lenguaje  atención cliente  \
0                       1                                1                 0   
1                       0                                0                 0   
2                       0                                0                 0   
3                       0                                0                 0   
4                       0                                0                 1   

   bert gpt  bert gpt cambiar  chatbot inteligente  \
0         0                 0                    0   
1         1                 1                    0   
2         0                 0                    0   
3         0                 0                    0   
4         0                 0                    1   

   chatbot inteligente atención  clase politécnico  \
0                             0                  0   
1                             0                  0   
2                             0                  1   
3                             0                  0   
4                             1                  0   

   clase politécnico interesante  crear chatbot  ...  modelo bert gpt  \
0                              0              0  ...                0   
1                              0              0  ...                1   
2                              1              0  ...                0   
3                              0              0  ...                0   
4                              0              1  ...                0   

   politécnico interesante  politécnico interesante desafiante  \
0                        0                                   0   
1                        0                                   0   
2                        1                                   1   
3                        0                                   0   
4                        0                                   0   

   procesamiento lenguaje  procesamiento lenguaje natural  querer crear  \
0                       1                               1             0   
1                       0                               0             0   
2                       0                               0             0   
3                       0                               0             0   
4                       0                               0             1   

   querer crear chatbot  técnico clave  vectorización técnico  \
0                     0              0                      0   
1                     0              0                      0   
2                     0              0                      0   
3                     0              1                      1   
4                     1              0                      0   

   vectorización técnico clave  
0                            0  
1                            0  
2                            0  
3                            1  
4                            0  

[5 rows x 31 columns]

## Vectorización y Representación del Texto

En esta sección se aplicaron diferentes técnicas de vectorización para convertir el texto procesado en representaciones numéricas, útiles para tareas de aprendizaje automático.

### Técnicas aplicadas:

- **Bag of Words (BoW)**: Cuenta la frecuencia de cada palabra en los textos. Simple pero efectiva en muchos contextos.
- **TF-IDF (Term Frequency - Inverse Document Frequency)**: Ajusta las frecuencias según qué tan únicas son las palabras en el corpus, penalizando las más comunes.
- **N-gramas (bi-gramas y tri-gramas)**: Captura secuencias de 2 y 3 palabras consecutivas para conservar algo de estructura gramatical.

### Mejoras implementadas:

- Se reutilizó el corpus **previamente lematizado** y limpio (`lemmatized_corpus`), lo que mejora notablemente la calidad de las representaciones.

Estas representaciones son fundamentales para alimentar modelos de clasificación, agrupamiento o detección de similitudes en el lenguaje natural.


# 4. Tareas Avanzadas con spaCy

In [18]:
from spacy import displacy

In [19]:
# Aplicar POS y NER sobre cada frase del corpus
for i, sentence in enumerate(corpus):
    doc = nlp(sentence)
    print(f"\nFrase {i+1}: {sentence}")
    print("Etiquetado POS:", [(token.text, token.pos_) for token in doc])
    print("Entidades reconocidas:", [(ent.text, ent.label_) for ent in doc.ents])


Frase 1: me encanta aprender procesamiento de lenguaje natural.
Etiquetado POS: [('me', 'PRON'), ('encanta', 'VERB'), ('aprender', 'VERB'), ('procesamiento', 'NOUN'), ('de', 'ADP'), ('lenguaje', 'NOUN'), ('natural', 'ADJ'), ('.', 'PUNCT')]
Entidades reconocidas: []

Frase 2: los modelos como bert y gpt han cambiado todo.
Etiquetado POS: [('los', 'DET'), ('modelos', 'NOUN'), ('como', 'SCONJ'), ('bert', 'PROPN'), ('y', 'CCONJ'), ('gpt', 'PROPN'), ('han', 'AUX'), ('cambiado', 'VERB'), ('todo', 'PRON'), ('.', 'PUNCT')]
Entidades reconocidas: [('bert', 'MISC'), ('gpt', 'ORG')]

Frase 3: esta clase del politécnico es muy interesante y desafiante.
Etiquetado POS: [('esta', 'DET'), ('clase', 'NOUN'), ('del', 'ADP'), ('politécnico', 'NOUN'), ('es', 'AUX'), ('muy', 'ADV'), ('interesante', 'ADJ'), ('y', 'CCONJ'), ('desafiante', 'ADJ'), ('.', 'PUNCT')]
Entidades reconocidas: []

Frase 4: la lematización y la vectorización son técnicas clave.
Etiquetado POS: [('la', 'DET'), ('lematización', 'NOUN'

In [20]:
# Dependencias sintácticas
displacy.render(doc, style='dep', jupyter=True)

In [23]:

# Limitaciones de Bolsa de Palabras
print("BoW no captura el significado de las palabras ni el orden de las mismas.")

BoW no captura el significado de las palabras ni el orden de las mismas.


## Tareas Avanzadas con spaCy

En esta sección se aplicaron técnicas de análisis lingüístico avanzado utilizando el modelo de lenguaje en español de `spaCy`.

### Procesos realizados:

- **Etiquetado gramatical (POS Tagging)**:  
  Cada palabra de las frases del corpus fue etiquetada con su categoría gramatical (verbo, sustantivo, adjetivo, etc.).

- **Reconocimiento de Entidades Nombradas (NER)**:  
  Se identificaron entidades como nombres de lugares, personas, organizaciones o fechas presentes en las frases.

- **Análisis de dependencias sintácticas**:  
  Se visualizó la estructura gramatical de una oración del corpus utilizando `displacy`.  
  Este análisis permite comprender la relación jerárquica y funcional entre las palabras de la oración.

### Mejora aplicada:

A diferencia del código original que analizaba solo una frase fija, aquí se procesaron todas las frases del nuevo corpus. Esto permitió extender el análisis a un conjunto realista y más relevante de datos.

La visualización sintáctica evidenció cómo spaCy es capaz de capturar no solo las palabras, sino también cómo se relacionan entre sí en la estructura de la oración, lo cual es clave para tareas complejas como resumen automático, generación de texto y análisis semántico.


## Conclusión Final

En esta actividad se realizó un análisis, refactorización y mejora del código original de procesamiento de lenguaje natural.

A lo largo del trabajo se implementaron mejoras significativas en la calidad del preprocesamiento textual y la coherencia del análisis aplicado. Se reemplazaron frases genéricas por un corpus más relevante y temático, se unificó el procesamiento en torno a `spaCy`, y se extendieron las técnicas a todos los textos.

Las mejoras principales incluyen:

- Uso de corpus propio temático.
- Lematización precisa con `spaCy`.
- Vectorización con corpus preprocesado (BoW, TF-IDF y N-gramas).
- Análisis sintáctico y reconocimiento de entidades más amplio.
- Explicaciones claras con Markdown y secciones bien documentadas.

Este trabajo no solo refuerza los conceptos básicos de NLP, sino que también demuestra cómo aplicar herramientas modernas para construir una pipeline textual robusta y extensible.

La notebook final está lista para ser utilizada como base para proyectos más avanzados en procesamiento de texto en español.
